In [ ]:
!git clone https://github.com/declare-lab/conv-emotion

Cloning into 'conv-emotion'...
remote: Enumerating objects: 1388, done.
remote: Counting objects: 100% (324/324), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 1388 (delta 156), reused 218 (delta 91), pack-reused 1064
Receiving objects: 100% (1388/1388), 751.31 MiB | 30.02 MiB/s, done.
Resolving deltas: 100% (700/700), done.
Checking out files: 100% (435/435), done.


In [ ]:
import pickle

_, videoSpeakers, videoLabels, _, _, _, videoSentence, trainVid, testVid = pickle.load(open("/content/drive/MyDrive/Depot_Mouhamed-20220228T100527Z-001/DialogueRNN/DialogueRNN/IEMOCAP_features/IEMOCAP_features_raw.pkl", 'rb'), encoding='latin1')



In [ ]:
print(len(list(videoSentence.values())[0]))
print(len(list(videoLabels.values())[0]))

42
42


In [ ]:
%cd /content/conv-emotion/TL-ERC/

/content/conv-emotion/TL-ERC


In [ ]:
!python setup.py

In [ ]:
!cp /content/drive/MyDrive/Depot_Mouhamed-20220228T100527Z-001/DialogueRNN/DialogueRNN/IEMOCAP_features/IEMOCAP_features_raw.pkl /content/conv-emotion/TL-ERC/datasets

In [ ]:
!gdown --id 1OXtnyJ5nDMmK75L9kEQvKPIyO0xzyeVC


Downloading...
From: https://drive.google.com/uc?id=1OXtnyJ5nDMmK75L9kEQvKPIyO0xzyeVC
To: /content/conv-emotion/TL-ERC/bert_model/cornell_weights.pkl
100% 42.0M/42.0M [00:00<00:00, 137MB/s]


In [ ]:
%cd bert_model

/content/conv-emotion/TL-ERC/bert_model


In [ ]:
!python train.py --load_checkpoint=../generative_weights/cornell_weights.pkl --data=iemocap

iemocap
iemocap
iemocap
Configurations
{'activation': 'Tanh',
 'batch_size': 1,
 'bidirectional': True,
 'checkpoint': None,
 'clip': 1.0,
 'context_size': 256,
 'conversation_length_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/conversation_length.pkl'),
 'data': 'iemocap',
 'data_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train'),
 'dataset_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap'),
 'dropout': 0.0,
 'embedding_size': 300,
 'encoder_hidden_size': 768,
 'eval_batch_size': 2,
 'feedforward': 'FeedForward',
 'id2word_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/id2word.pkl'),
 'label_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/labels.pkl'),
 'learning_rate': 0.0001,
 'load_checkpoint': '../generative_weights/cornell_weights.pkl',
 'minimum_improvement': 0.001,
 'mode': 'train',
 'model': 'bc_RNN',
 'n_epoch': 500,
 'num_bert_layers': 4,
 'num_classes': 6,
 'num_layers': 1,
 'o

In [ ]:
!gdown --id 1nufbrBJ-LtcROv1MviCHFI7tQE3JnqQR

Downloading...
From: https://drive.google.com/uc?id=1nufbrBJ-LtcROv1MviCHFI7tQE3JnqQR
To: /content/conv-emotion/TL-ERC/bert_model/iemocap.zip
100% 14.2M/14.2M [00:00<00:00, 225MB/s]


In [ ]:
!unzip /content/conv-emotion/TL-ERC/bert_model/iemocap.zip -d /content/conv-emotion/TL-ERC/datasets

Archive:  /content/conv-emotion/TL-ERC/bert_model/iemocap.zip
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/._.DS_Store  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/.DS_Store  
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/test/
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/IEMOCAP_features_raw.pkl  
   creating: /content/conv-emotion/TL-ERC/datasets/iemocap/train/
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/video_id.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/sentence_length.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/sentences.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/labels.pkl  
  inflating: /content/conv-emotion/TL-ERC/datasets/iemocap/valid/conversation_length.pkl  
  inflating: /content/conv-emotion

In [ ]:
!pip install pytorch_pretrained_bert

In [ ]:
!pip install tensorboardX

In [ ]:
from solver import *
from data_loader import get_loader
from configs import get_config
from util import Vocab
import os
import pickle


def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


config = get_config(parse = False, mode='train')
val_config = get_config(parse = False, mode='valid')
test_config = get_config(parse = False, mode='test')

# _RUNS = config.runs

# _best_test_loss, _best_test_f1_w, _best_test_f1_m, _best_epoch = [], [], [], []

# for run in range(_RUNS):

print(config)

# No. of videos to consider
training_data_len = int(config.training_percentage * \
    len(load_pickle(config.sentences_path)))


train_data_loader = get_loader(
    sentences=load_pickle(config.sentences_path)[:training_data_len],
    labels=load_pickle(config.label_path)[:training_data_len],
    conversation_length=load_pickle(config.conversation_length_path)[:training_data_len],
    sentence_length=load_pickle(config.sentence_length_path)[:training_data_len],
    batch_size=config.batch_size)

eval_data_loader = get_loader(
    sentences=load_pickle(val_config.sentences_path),
    labels=load_pickle(val_config.label_path),
    conversation_length=load_pickle(val_config.conversation_length_path),
    sentence_length=load_pickle(val_config.sentence_length_path),
    batch_size=val_config.eval_batch_size,
    shuffle=False)

test_data_loader = get_loader(
    sentences=load_pickle(test_config.sentences_path),
    labels=load_pickle(test_config.label_path),
    conversation_length=load_pickle(test_config.conversation_length_path),
    sentence_length=load_pickle(test_config.sentence_length_path),
    batch_size=test_config.eval_batch_size,
    shuffle=False)




# for testing
solver = Solver(config, train_data_loader,
                eval_data_loader, test_data_loader, is_train=True)


solver.build()

best_test_loss, best_test_f1_w, best_epoch = solver.train()

    # print(f"Current RUN: {run+1}")

    # print("\n\nBest test loss")
    # print(best_test_loss)
    # print("Best test f1 weighted")
    # print(best_test_f1_w)
    # print("Best epoch")
    # print(best_epoch)

    # _best_test_loss.append(best_test_loss)
    # _best_test_f1_w.append(best_test_f1_w)
    # _best_epoch.append(best_epoch)


# Print final
# print(f"\n\nAverage across runs:")

# print("Best epoch")
# print(_best_epoch)

# print("\n\nBest test loss")
# print(np.mean(np.array(_best_test_loss), axis=0))

# print("Overall test f1 weighted")
# print(np.array(_best_test_f1_w))

# print("Best test f1 weighted")
# print(np.mean(np.array(_best_test_f1_w), axis=0))

iemocap
iemocap
iemocap
Configurations
{'activation': 'Tanh',
 'batch_size': 1,
 'bidirectional': True,
 'checkpoint': None,
 'clip': 1.0,
 'context_size': 256,
 'conversation_length_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/conversation_length.pkl'),
 'data': 'iemocap',
 'data_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train'),
 'dataset_dir': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap'),
 'dropout': 0.0,
 'embedding_size': 300,
 'encoder_hidden_size': 768,
 'eval_batch_size': 2,
 'feedforward': 'FeedForward',
 'id2word_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/id2word.pkl'),
 'label_path': PosixPath('/content/conv-emotion/TL-ERC/datasets/iemocap/train/labels.pkl'),
 'learning_rate': 0.0001,
 'load_checkpoint': None,
 'minimum_improvement': 0.001,
 'mode': 'train',
 'model': 'bc_RNN',
 'n_epoch': 500,
 'num_bert_layers': 4,
 'num_classes': 6,
 'num_layers': 1,
 'optimizer': <class 'torch.optim.adam.Ada

  1%|▍                                           | 1/96 [00:00<00:55,  1.71it/s]

Epoch: 1, iter 0: loss = 1.9334471225738525


100%|███████████████████████████████████████████| 96/96 [00:32<00:00,  2.93it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

Epoch 1 loss average: 1.800
train
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       376
           1     0.2175    0.2147    0.2161       764
           2     0.2360    0.1796    0.2040      1080
           3     0.1271    0.1001    0.1120       749
           4     0.0521    0.0192    0.0281       520
           5     0.2768    0.5355    0.3650      1210

    accuracy                         0.2322      4699
   macro avg     0.1516    0.1749    0.1542      4699
weighted avg     0.1869    0.2322    0.1970      4699



...

  1%|▍                                           | 1/96 [00:00<00:25,  3.78it/s]

Epoch: 10, iter 0: loss = 0.28646618127822876


100%|███████████████████████████████████████████| 96/96 [00:33<00:00,  2.88it/s]


Epoch 10 loss average: 0.418
train
              precision    recall  f1-score   support

           0     0.8054    0.7154    0.7577       376
           1     0.8619    0.8901    0.8757       764
           2     0.8402    0.8324    0.8363      1080
           3     0.8311    0.8411    0.8361       749
           4     0.8120    0.8558    0.8333       520
           5     0.8408    0.8339    0.8373      1210

    accuracy                         0.8368      4699
   macro avg     0.8319    0.8281    0.8294      4699
weighted avg     0.8365    0.8368    0.8363      4699



KeyboardInterrupt: ignored

## Evaluation

In [ ]:
validation_loss, w_valid_f1, valid_predictions = solver.evaluate(eval_data_loader, mode="valid")


In [ ]:
labels = [e[1] for e in eval_data_loader]
Labels = [label for utt in labels for label in utt]
Labels = [e for k in Labels for e in k]

In [ ]:
solver.print_metric(Labels, valid_predictions, "test")

test
              precision    recall  f1-score   support

           0     0.4028    0.4531    0.4265       128
           1     0.2951    0.7200    0.4186        75
           2     0.3946    0.3607    0.3769       244
           3     0.6181    0.4837    0.5427       184
           4     0.5828    0.3964    0.4718       222
           5     0.4962    0.5116    0.5038       258

    accuracy                         0.4581      1111
   macro avg     0.4649    0.4876    0.4567      1111
weighted avg     0.4870    0.4581    0.4613      1111



0.4613229695600036

## Inference

In [ ]:
%cd ..

/content/conv-emotion/TL-ERC


In [ ]:
from iemocap_preprocess import *

Loading SpaCy


In [ ]:
conv = videoSentence['Ses01F_impro01']
conv

['Excuse me.',
 'Do you have your forms?',
 'Yeah.',
 'Let me see them.',
 'Is there a problem?',
 'Who told you to get in this line?',
 "Okay. But I didn't tell you to get in this line if you are filling out this particular form.",
 "Well what's the problem?  Let me change it.",
 'This form is a Z.X.four.',
 "You can't--  This is not the line for Z.X.four.  If you're going to fill out the Z.X.four, you need to have a different form of ID.",
 "What?  I'm getting an ID.  This is why I'm here.  My wallet was stolen.",
 'No. I need another set of ID to prove this is actually you.',
 'How am I supposed to get an ID without an ID?  How does a person get an ID in the first place?',
 "I don't know.  But I need an ID to pass this form along.  I can't just send it along without an ID.",
 "I'm here to get an ID.",
 'No.  I need another ID, a separate one.',
 'Like what?  Like a birth certificate?',
 "A birth certificate, a passport...a student ID; didn't you go to school?  Anything?",
 "Yes but 

In [ ]:
labels = videoLabels['Ses01F_impro01']
labels

[2, 5, 2, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 2, 3, 5]

In [ ]:


parser = argparse.ArgumentParser()
    
# Load the dataset
iemocap = IEMOCAP()
iemocap.load_iemocap_data()

# Maximum valid length of sentence
# => SOS/EOS will surround sentence (EOS for source / SOS for target)
# => maximum length of tensor = max_sentence_length + 1
parser.add_argument('-s', '--max_sentence_length', type=int, default=30)

# Vocabulary
parser.add_argument('--max_vocab_size', type=int, default=20000)
parser.add_argument('--min_vocab_frequency', type=int, default=5)

args = parser.parse_args(args=[])

max_sent_len = args.max_sentence_length
max_conv_len = iemocap.max_conv_length
max_vocab_size = args.max_vocab_size
min_freq = args.min_vocab_frequency


conv_sentences = list([tokenize_conversation(conv)])
conversation_length = [min(len(conv), max_conv_len)]

conv_labels = [labels]
# fix labels as per conversation_length
for idx, conv_len in enumerate(conversation_length):
    conv_labels[idx]=conv_labels[idx][:conv_len]


sentences, sentence_length = pad_sentences(
    conv_sentences,
    max_sentence_length=max_sent_len,
    max_conversation_length=max_conv_len)

for sentence_len, label in zip(conversation_length, conv_labels):
    assert(sentence_len ==len(label))


In [ ]:
sentences

In [ ]:
labels

[2, 5, 2, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 2, 3, 5]

In [ ]:
infer_loader = get_loader(
    sentences,
    [[0 for s in conv] for conv in sentences ],
    conversation_length=conversation_length,
    sentence_length=sentence_length,
    batch_size=val_config.eval_batch_size,
    shuffle=False)

In [ ]:
infer_loader

In [ ]:
def inference(conversations, labels, conversation_length, sentence_length, type_ids, masks):
  # conversations: (batch_size) list of conversations
  #   conversation: list of sentences
  #   sentence: list of tokens
  # conversation_length: list of int
  # sentence_length: (batch_size) list of conversation list of sentence_lengths

  input_conversations = conversations

  # flatten input and target conversations
  input_sentences = [sent for conv in input_conversations for sent in conv]
  input_labels = [label for conv in labels for label in conv]
  input_sentence_length = [l for len_list in sentence_length for l in len_list]
  input_conversation_length = [l for l in conversation_length]
  input_masks = [mask for conv in masks for mask in conv]
  orig_input_labels = input_labels

  with torch.no_grad():
      # transfering the input to cuda
      input_sentences = to_var(torch.LongTensor(input_sentences))
      input_labels = to_var(torch.LongTensor(input_labels))
      input_sentence_length = to_var(torch.LongTensor(input_sentence_length))
      input_conversation_length = to_var(torch.LongTensor(input_conversation_length))
      input_masks = to_var(torch.LongTensor(input_masks))

  sentence_logits = solver.model(
      input_sentences,
      input_sentence_length,
      input_conversation_length,
      input_masks)

  present_predictions = list(np.argmax(sentence_logits.detach().cpu().numpy(), axis=1))
  print(present_predictions)

In [ ]:
inference(*next(iter(infer_loader)))

[2, 5, 2, 5, 2, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 2, 3, 5]


In [ ]:
import torch
import numpy as np

from util import to_var

In [ ]:
input = next(iter(eval_data_loader))
print(np.array(input[2],dtype=object ).shape)

(2,)


In [ ]:
sentences=load_pickle(val_config.sentences_path)

In [ ]:
l2i = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger':6}
i2l = {v:k for k,v in l2i.items()}

In [ ]:
conv_id = 1
for replique,label in zip(sentences[conv_id],input[1][conv_id]):

  print(" ".join(replique))
  print(i2l[label]) 


In [ ]:
# Labels, sentences
labels=load_pickle(val_config.label_path)
sentences=load_pickle(val_config.sentences_path)

In [ ]:
labels = [[3,
 2,
 2,
 3]]

In [ ]:
sentences = [[['why',
  'did',
  'he',
  'invite',
  'her',
  'here',
  '?',
  '<eos>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['oh',
  ',',
  'god',
  ',',
  'line',
  '.',
  '<eos>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['why',
  'does',
  'that',
  'bother',
  'you',
  '?',
  '<eos>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>'],
 ['she',
  "'s",
  'been',
  'in',
  'new',
  'york',
  'three',
  'and',
  'a',
  'half',
  'years',
  '.',
  'why',
  ',',
  'all',
  'of',
  'a',
  'sudden',
  '<eos>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>',
  '<pad>']]]

In [ ]:
load_pickle(val_config.sentence_length_path)